In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
#pulling data API
key =credentials.login['key_isaham']  # extract password

### 1) Extract stock data from iSaham API
### 2) Run Sthocastic function on each counter

In [2]:
# Define the instruments to download.
#tech = ["GREATEC", 
       #"INARI"]

tech = ["GREATEC",
        "UWC",       
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ]

       
# looping through all stocks
# dictionary of dataframes
d = {}
# listing all stocks
all_stocks = []       


for m,stock in enumerate (tech):
        
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []


    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
        #print("Position closed")
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    
    trans['Stock'] = pd.Series(Name_stock)
    
    
    
    # saving in multiple df
    d["{}".format(stock)] = trans
    
    # list of all stocks
    all_stocks.append(d[stock])
    
    
    #print(trans)
    
    #dct_1={ 'Number of stock':dct
    
    #}
    #trans_1 = pd.DataFrame(dct_1)
    
    #print(trans[0:])
    
    #trans.to_csv('result {}.csv'.format(symbol), index=False)
    #C:\Users\Mo Syamil\Documents\data scientiest trial\strategy stho\result
            
    print("----------------DONE {}------------------".format(symbol))
    #print(trans)
    #concat_all= pd.concat([trans], axis=0)
    #print(concat_all)
    #data_1=[]
    
    #for d in  range(len(trans)):
        #concat_all=pd.concat([trans], axis=0)
        #data_1.append(list(trans))
        #print(data_1)
    #c_data= trans[m]
    #print(concat_all)
    #print(trans)
    
    
        
    #print(trans)
    #print()

    
# Merging all df
#concat_all= pd.concat([trans], axis=0)
#print(trans.append(m))
#print(trans)
#print(concat_all)

#print("Finish")

    



----------------PROCESSING GREATEC------------------
----------------DONE GREATEC------------------
----------------PROCESSING UWC------------------
----------------DONE UWC------------------
----------------PROCESSING D&O------------------
----------------DONE D&O------------------
----------------PROCESSING GENETEC------------------
----------------DONE GENETEC------------------
----------------PROCESSING UNISEM------------------
----------------DONE UNISEM------------------
----------------PROCESSING FRONTKN------------------
----------------DONE FRONTKN------------------
----------------PROCESSING ATAIMS------------------
----------------DONE ATAIMS------------------
----------------PROCESSING FPI------------------
----------------DONE FPI------------------
----------------PROCESSING PENTA------------------
----------------DONE PENTA------------------
----------------PROCESSING DUFU------------------
----------------DONE DUFU------------------
----------------PROCESSING VS---------

In [ ]:
#list of all stocks

In [3]:
df_all=pd.concat(list(d.values()),ignore_index=True)

In [4]:
df_all

,Buy_Date,Buy,Sell,Sell_Date,Open,Volume,Low,high,Pct_change,wr,Stock
0,2019-11-25,0.960,1.095,2019-12-09,0.960,3698600.0,0.945,0.965,14.062500,1,GREATEC
1,2019-11-27,0.935,1.095,2019-12-09,0.955,6644600.0,0.915,0.970,17.112299,1,GREATEC
2,2020-03-10,1.500,1.290,2020-04-15,1.325,10114800.0,1.310,1.525,-14.000000,0,GREATEC
3,2020-09-11,3.000,3.340,2020-10-02,2.900,6414400.0,2.815,3.035,11.333333,1,GREATEC
4,2020-09-18,3.075,3.340,2020-10-02,3.000,937200.0,2.990,3.075,8.617886,1,GREATEC
...,...,...,...,...,...,...,...,...,...,...,...
1340,2020-05-21,8.000,9.150,2020-06-29,7.960,181800.0,7.960,8.200,14.375000,1,VITROX
1341,2020-09-11,12.260,13.800,2020-10-20,11.700,504800.0,11.660,12.260,12.561175,1,VITROX
1342,2020-11-27,14.900,15.540,2021-01-19,14.520,113500.0,14.520,15.200,4.295302,1,VITROX
1343,2021-02-24,17.200,16.200,2021-04-05,17.020,163900.0,16.860,17.200,-5.813953,0,VITROX
